In [21]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import csv
import os
import matplotlib as mpl

path = "../../extern/larnest/utils/"


In [24]:
class LArDataset:
    """
    """
    def __init__(self,
        dataset_file: str=path+'../data/lar_data.npz',
        benchmarks_file: str='',
        plot_config: dict={},
        fluctuations: bool=False,
    ):
        self.dataset_file = dataset_file
        self.benchmarks_file = benchmarks_file
        self.plot_config = plot_config
        self.fluctuations = fluctuations
        # set up datasets
        self.data = dict(np.load(self.dataset_file, allow_pickle=True))
        self.datasets = {key: self.data[key].item() for key in self.data}

        self.benchmarks = pd.read_csv(
            self.benchmarks_file,
            names=[
                'type','energy', 'efield' , 
                'TotalYields', 'QuantaYields', 'LightYields', 
                'Nph' , 'Ne', 'Nex', 'Nion',
                'TotalYields_std', 'QuantaYields_std', 'LightYields_std',
                'Nph_std', 'Ne_std', 'Nex_std', 'Nion_std',
            ],
            header=0,
            #dtype=float
        )
        self.benchmarks.replace([np.inf, -np.inf, np.nan], 0.0)
        self.ylabels = {
            'nr_total':  r"$Y_q$ [quanta/keV]",
            'nr_charge': r"$Y_{e^-}$ [thermal electron/keV]",
            'nr_light':  r"$Y_{\gamma}$ [optical photon/keV]",
            'nr_nph':    r"$\langle N_{ph}\rangle$ [optical photon]",
            'nr_ne':     r"$\langle N_{e^{-}}\rangle$ [thermal electron]",
            'nr_nex':    r"$\langle N_{ex}\rangle$ [exciton]",
            'nr_nion':   r"$\langle N_{ion}\rangle$ [ion]",
            'er_total':  r"$Y_q$ [total quanta/keV]",
            'er_charge': r"$Y_{e^-}$ [thermal electron/keV]",
            'er_light':  r"$Y_{\gamma}$ [optical photon/keV]",
            'er_nph':    r"$\langle N_{ph}\rangle$ [optical photon]",
            'er_ne':     r"$\langle N_{e^{-}}\rangle$ [thermal electron]",
            'er_nex':    r"$\langle N_{ex}\rangle$ [exciton]",
            'er_nion':   r"$\langle N_{ion}\rangle$ [ion]",
            'alpha_total':  r"$Y_q$ [total quanta/keV]",
            'alpha_charge': r"$Y_{e^-}$ [thermal electron/keV]",
            'alpha_light':  r"$Y_{\gamma}$ [optical photon/keV]",
            'alpha_nph':    r"$\langle N_{ph}\rangle$ [optical photon]",
            'alpha_ne':     r"$\langle N_{e^{-}}\rangle$ [thermal electron]",
            'alpha_nex':    r"$\langle N_{ex}\rangle$ [exciton]",
            'alpha_nion':   r"$\langle N_{ion}\rangle$ [ion]",
        }
        self.xlabels = {
            'nr_total':  "Nuclear Recoil Energy Loss [keV]",
            'nr_charge': "Nuclear Recoil Energy Loss [keV]",
            'nr_light':  "Nuclear Recoil Energy Loss [keV]",
            'nr_nph':    "Nuclear Recoil Energy Loss [keV]",
            'nr_ne':     "Nuclear Recoil Energy Loss [keV]",
            'nr_nex':    "Nuclear Recoil Energy Loss [keV]",
            'nr_nion':   "Nuclear Recoil Energy Loss [keV]",
            'er_total':  "Electron Recoil Energy Loss [keV]",
            'er_charge': "Electron Recoil Energy Loss [keV]",
            'er_light':  "Electron Recoil Energy Loss [keV]",
            'er_nph':    "Electron Recoil Energy Loss [keV]",
            'er_ne':     "Electron Recoil Energy Loss [keV]",
            'er_nex':    "Electron Recoil Energy Loss [keV]",
            'er_nion':   "Electron Recoil Energy Loss [keV]",
            'alpha_total':  "Alpha Recoil Energy Loss [keV]",
            'alpha_charge': "Alpha Recoil Energy Loss [keV]",
            'alpha_light':  "Alpha Recoil Energy Loss [keV]",
            'alpha_nph':    "Alpha Recoil Energy Loss [keV]",
            'alpha_ne':     "Alpha Recoil Energy Loss [keV]",
            'alpha_nex':    "Alpha Recoil Energy Loss [keV]",
            'alpha_nion':   "Alpha Recoil Energy Loss [keV]",
        }
        self.titles = {
            'nr_total':  "LArNEST Nuclear Recoil Total Yields",
            'nr_charge': "LArNEST Nuclear Recoil Charge Yields",
            'nr_light':  "LArNEST Nuclear Recoil Light Yields",
            'nr_nph':    "LArNEST Nuclear Recoil Photon Yields",
            'nr_ne':     "LArNEST Nuclear Recoil Electron Yields",
            'nr_nex':    "LArNEST Nuclear Recoil Exciton Yields",
            'nr_nion':   "LArNEST Nuclear Recoil Ion Yields",
            'er_total':  "LArNEST Electron Recoil Total Yields",
            'er_charge': "LArNEST Electron Recoil Charge Yields",
            'er_light':  "LArNEST Electron Recoil Light Yields",
            'er_nph':    "LArNEST Electron Recoil Photon Yields",
            'er_ne':     "LArNEST Electron Recoil Electron Yields",
            'er_nex':    "LArNEST Electron Recoil Exciton Yields",
            'er_nion':   "LArNEST Electron Recoil Ion Yields",
            'alpha_total':  "LArNEST Alpha Total Yields",
            'alpha_charge': "LArNEST Alpha Charge Yields",
            'alpha_light':  "LArNEST Alpha Light Yields",
            'alpha_nph':    "LArNEST Alpha Photon Yields",
            'alpha_ne':     "LArNEST Alpha Electron Yields",
            'alpha_nex':    "LArNEST Alpha Exciton Yields",
            'alpha_nion':   "LArNEST Alpha Ion Yields",
        }
        self.benchmark_types = {
            'nr_total':  ['NR','TotalYields'],
            'nr_charge': ['NR','QuantaYields'],
            'nr_light':  ['NR','LightYields'],
            'nr_nph':    ['NR','Nph'],
            'nr_ne':     ['NR','Ne'],
            'nr_nex':    ['NR','Nex'],
            'nr_nion':   ['NR','Nion'],
            'er_total':  ['ER','TotalYields'],
            'er_charge': ['ER','QuantaYields'],
            'er_light':  ['ER','LightYields'],
            'er_nph':    ['ER','Nph'],
            'er_ne':     ['ER','Ne'],
            'er_nex':    ['ER','Nex'],
            'er_nion':   ['ER','Nion'],
            'alpha_total':  ['Alpha','TotalYields'],
            'alpha_charge': ['Alpha','QuantaYields'],
            'alpha_light':  ['Alpha','LightYields'],
            'alpha_nph':    ['Alpha','Nph'],
            'alpha_ne':     ['Alpha','Ne'],
            'alpha_nex':    ['Alpha','Nex'],
            'alpha_nion':   ['Alpha','Nion'],
        }

        self.fluctuation_types = [
            'Nph', 'Ne', 'Nex', 'Nion'
        ]

        # create plotting directory
        if not os.path.isdir(path+"../plots/mean_yields/"):
            os.makedirs(path+"../plots/mean_yields")
        if not os.path.isdir(path+"../plots/fluctuations/"):
            os.makedirs(path+"../plots/fluctuations")
        if not os.path.isdir(path+"../plots/data/"):
            os.makedirs(path+"../plots/data")
        if not os.path.isdir(path+"../plots/data_fluctuations/"):
            os.makedirs(path+"../plots/data_fluctuations")

    def plot_mean_yields(self, dataset_type: str='nr_total'):
        # Set font and enable LaTeX for text rendering
        mpl.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
        mpl.rc('text', usetex=True)

        fig, axs = plt.subplots(figsize=(8, 6))
        
        # Color mapping
        colors = ['#E41A1C', '#377EB8', '#4DAF4A', '#FF7F00', '#984EA3', '#A65628', '#F781BF', '#999999']
        benchmark_mask = (self.benchmarks['type'] == self.benchmark_types[dataset_type][0])
        unique_efields = np.unique(self.benchmarks['efield'][benchmark_mask])
        color_mapping = {efield: colors[i % len(colors)] for i, efield in enumerate(unique_efields)}

        # Plotting the data
        for efield in unique_efields:
            combined_mask = benchmark_mask & (self.benchmarks['efield'] == efield)
            energy = self.benchmarks['energy'][combined_mask].values
            yields = self.benchmarks[self.benchmark_types[dataset_type][1]][combined_mask].values
            yields[yields < 0] = 0

            color = color_mapping[efield]
            axs.plot(energy, yields, color=color, marker='o', markeredgewidth=0.5,
                    markerfacecolor=color, markeredgecolor=color, ms=2, lw=0.8, 
                    label=f"NEST - {efield} V/cm")

        # Axis and scale settings
        axs.set_xscale("log")
        if "_n" in dataset_type:
            axs.set_yscale("log")
        axs.tick_params(which='both', direction='in', top=True, right=True, length=4, labelsize=10)
        axs.tick_params(which='minor', length=2)
        axs.minorticks_on()
        axs.grid(which='both', linestyle=':', linewidth=0.5, color='gray', alpha=0.5)

        # Legend, title, and labels
        leg = axs.legend(frameon=False, loc='best', handlelength=1, handleheight=1)
        for handle in leg.legendHandles:
            handle.set_alpha(1.0) 
        axs.set_xlabel(self.xlabels[dataset_type], fontsize=12)
        axs.set_ylabel(self.ylabels[dataset_type], fontsize=12)
        
        plt.tight_layout()
        plt.savefig(f"../../extern/larnest/utils/../plots/mean_yields/{self.benchmark_types[dataset_type][0]}_{self.benchmark_types[dataset_type][1]}.png")
        plt.close()

            
    def plot_all_yields(self,
        ):
            for benchmark_type in self.benchmark_types:
                self.plot_mean_yields(benchmark_type)



In [25]:
dataset_file = path+"../data/lar_data.npz"
mean_yields_benchmarks_file = path+"../data/mean_yields_benchmarks.csv"
fluctuations_benchmarks_file = path+"../data/fluctuation_benchmarks.csv"

lar_datasets = []
lar_dataset = LArDataset(
        dataset_file=dataset_file,
        benchmarks_file=mean_yields_benchmarks_file,
        fluctuations=False,
    )

lar_dataset.plot_all_yields()
